In [1]:
import requests
import json
import pandas as pd

import time

In [2]:
def get_replays(num_replays, params, auth_key, collected_replays = 0, id_list = []):
    time.sleep(0.3)
    res = requests.get('https://ballchasing.com/api/replays/',
                  headers={
                      'Authorization': auth_key},
                  params = params
                  )
    data = res.json()
    collected_replays += len(data['list'])
    print(collected_replays, num_replays)
    if collected_replays <= num_replays:
        for i in range(len(data['list'])):
            id_list.append(data['list'][i]['id'])
            replay_end_date = data['list'][-1]['date']
            params['replay-date-before'] = replay_end_date
            
                
        get_replays(num_replays = num_replays, 
                    params = params, 
                    auth_key = auth_key,
                    collected_replays = collected_replays,
                    id_list = id_list
                   )
                
    return id_list



def get_gamedata(id_list, auth_key, player_name = None):
    
    errors = 0
    
    res = requests.get(f'https://ballchasing.com/api/replays/{id_list[0]}',
                  headers={
                      'Authorization': auth_key}
                          )
    data = res.json()
    
    player_data = data['orange']['players'][0]['stats']
    player_data = dict(player_data, **player_data['core'])
    player_data = dict(player_data, **player_data['boost'])
    player_data = dict(player_data, **player_data['movement'])
    player_data = dict(player_data, **player_data['positioning'])
    player_data = dict(player_data, **player_data['demo'])
    del player_data['core']
    del player_data['boost']
    del player_data['movement']
    del player_data['positioning']
    del player_data['demo']

#     player_data['title'] = data['title']
    player_data['player_name'] = player_name
    
    df = pd.DataFrame(columns = player_data.keys())
    
    for i, s in enumerate(id_list):
        
        time.sleep(0.3)
        res = requests.get(f'https://ballchasing.com/api/replays/{s}',
                  headers={
                      'Authorization': auth_key}
                          )
        data = res.json()    
#         print(data['blue'])
        if 'orange' not in data.keys():
            print(f"match {i} failed to retrieve data, {df.shape[0]} games collected")
            errors += 1
            continue
        try:
            
            if len(data['orange']['players']) != 3:
                print(f"match {i} not 3V3")
                errors += 1
                continue
        except KeyError:
                errors += 1
                print(f"KeyError at match {i}, errors = {errors}")
                continue
        
        if df.shape[0] % 20 == 0:
            print(f"{df.shape[0]} games collected, {errors} errors")
                
        try:
            for j in range(0, min([len(data['orange']['players']), len(data['blue']['players'])])):
            
                if player_name != None:
                    if data['orange']['players'][j]['name'].lower() == player_name.lower():
                        player_data = data['orange']['players'][j]['stats']

                        player_data = dict(player_data, **player_data['core'])
                        player_data = dict(player_data, **player_data['boost'])
                        player_data = dict(player_data, **player_data['movement'])
                        player_data = dict(player_data, **player_data['positioning'])
                        player_data = dict(player_data, **player_data['demo'])
                        del player_data['core']
                        del player_data['boost']
                        del player_data['movement']
                        del player_data['positioning']
                        del player_data['demo']

                        player_data['title'] = data['title']
                        player_data['player_name'] = player_name
                        df.loc[player_data['title'], :] = player_data

                    elif data['blue']['players'][j]['name'].lower() == player_name.lower():
                        
                        player_data = data['blue']['players'][j]['stats']

                        player_data = dict(player_data, **player_data['core'])
                        player_data = dict(player_data, **player_data['boost'])
                        player_data = dict(player_data, **player_data['movement'])
                        player_data = dict(player_data, **player_data['positioning'])
                        player_data = dict(player_data, **player_data['demo'])
                        del player_data['core']
                        del player_data['boost']
                        del player_data['movement']
                        del player_data['positioning']
                        del player_data['demo']

                        player_data['title'] = data['title']
                        player_data['player_name'] = player_name

                        df.loc[player_data['title'], :] = player_data
                
        except KeyError:
            errors += 1
            print(f'KeyError in match {i}, errors = {errors}')
            

    return df, print(f"{df.shape[0]} games collected, errors = {errors}")

In [3]:
params = {            'player-name' : 'boay00',
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'replay-date-before' : '2023-04-19T15:00:05+01:00'
                  }

In [4]:
boay_id_list = []
boay_id_list = get_replays(num_replays=20, 
                      params = params, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5')

19 20
20 20
21 20


In [6]:
boay_id_list = boay_id_list[:-1]


In [27]:
boay_id_list

['d0971503-bba7-41a4-bfc8-129a3f479ce3',
 '305e79bd-9b27-40c6-b0b6-66a73063f3dd',
 '0d3d3bc6-f09b-4d77-9047-8afa863334ce',
 'ec051ec4-4ed7-4ed5-a630-2209e6d895c0',
 '36778dbd-048b-46e3-aef1-94a3895d2543',
 '4551c1c0-ccd0-4689-b256-225337fd27ac',
 '51ba1e3e-d26e-4d68-935b-d7d6b91f2ec1',
 '19f11308-c5fc-4c78-a9ca-8a85ef2f9caa',
 '17755f72-b306-4830-b223-815f7ab7cfa9',
 '2fc24b07-0b63-4616-93ec-187f2e276a63',
 '247e14f3-81b5-4331-b275-59d967abc83f',
 '9b9ee975-3202-4c97-836d-473107073a79',
 '5d2c0afc-2053-4a72-923f-99694f6214c6',
 'ee8f3004-3818-4878-a020-11473a6b4330',
 'ea6e57d9-70ef-498c-95d1-80b2e53be135',
 'ce37d5f9-b46c-4de2-b1b5-097fd4b9e73d',
 '90a28893-7b97-4a3b-94db-47d6a7c8dd7f',
 '64a2ddb8-b65d-435c-a4c5-9f3963f53a95',
 'd1041d90-6e45-4188-ae6b-838bc1340fc9']

In [8]:
boay00 = get_gamedata(id_list = boay_id_list, 
             auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5',
             player_name = 'boay00')

0 games collected, 0 errors
19 games collected, errors = 0


In [11]:
boay00 = boay00[0]

In [33]:
params = {            'player-id' : 'xbox:2f7e6c0100000900',
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'replay-date-before' : '2023-04-19T15:00:05+01:00'
                  }

In [34]:
cho_id_list = []
cho_id_list = get_replays(num_replays=20, 
                      params = params, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

19 20
20 20
21 20


In [36]:
cho_id_list = cho_id_list[:-1]

In [37]:
res = requests.get(f'https://ballchasing.com/api/replays/{cho_id_list[0]}',
                  headers={
                      'Authorization': 'mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5'})

data = res.json()
data

{'id': 'b7e5be3f-50ff-4ff5-ac29-0a020d0dc327',
 'link': 'https://ballchasing.com/api/replays/b7e5be3f-50ff-4ff5-ac29-0a020d0dc327',
 'created': '2023-02-25T02:34:43.286117Z',
 'uploader': {'steam_id': '76561198986981881',
  'name': 'ksaco.',
  'profile_url': 'https://steamcommunity.com/id/KsacoRL/',
  'avatar': 'https://avatars.akamai.steamstatic.com/56bfda0041d7289f707cca02df001b4e42a67a6d.jpg'},
 'status': 'ok',
 'rocket_league_id': '002F44CF4B533DCA36C0E5A0B013E0E2',
 'match_guid': 'FCD5ACE211EDB4B3DB3B6881555AFE8E',
 'title': '2023-02-24.21.34 Ksacoo Ranked Standard Win',
 'map_code': 'trainstation_night_p',
 'match_type': 'Online',
 'team_size': 3,
 'playlist_id': 'ranked-standard',
 'duration': 356,
 'overtime': False,
 'season': 9,
 'season_type': 'free2play',
 'date': '2023-02-24T21:34:40-05:00',
 'date_has_timezone': True,
 'visibility': 'public',
 'min_rank': {'id': 'champion-1',
  'tier': 16,
  'division': 4,
  'name': 'Champion I Division 4'},
 'max_rank': {'id': 'champion-

In [38]:
cho_shmo = get_gamedata(id_list = cho_id_list, 
             auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5',
             player_name = 'Cho Shmo')

0 games collected, 0 errors
19 games collected, errors = 0


In [39]:
cho_shmo = cho_shmo[0]

In [40]:
cho_shmo

,shots,shots_against,goals,goals_against,saves,assists,score,mvp,shooting_percentage,bpm,...,percent_offensive_half,percent_behind_ball,percent_infront_ball,percent_most_back,percent_most_forward,percent_closest_to_ball,percent_farthest_from_ball,inflicted,taken,player_name
2023-02-24.21.34 Ksacoo Ranked Standard Win,2,6,1,3,1,0,433,False,50,310,...,22.057592,68.74784,31.252163,43.697926,27.684217,31.141052,31.466053,1,3,Cho Shmo
2023-02-24.20.34 GUIN19 Ranked Standard Loss,2,6,1,3,1,0,433,False,50,310,...,22.057379,68.752525,31.247475,44.215073,28.155685,31.556845,32.26665,1,3,Cho Shmo
2022-12-17.14.21 Motiv Bowl Ranked Standard Win,1,8,0,4,0,0,144,False,0,308,...,30.328321,64.74412,35.255882,39.406185,30.951767,31.43897,39.750095,0,1,Cho Shmo
2022-12-16.15.52 aidan Ranked Standard Win,3,5,1,2,0,1,272,False,33.333332,257,...,38.743294,74.927986,25.072014,49.608124,27.936113,23.48296,47.292484,0,0,Cho Shmo
2022-12-03.18.55 Sir Squirt Ranked Standard Win,2,3,0,1,0,0,128,False,0,279,...,36.85913,72.40629,27.593714,34.75533,31.341309,27.496693,38.32313,1,2,Cho Shmo
2022-11-13.19.09 werdnA Ranked Standard Win,4,2,4,0,0,0,581,True,100,295,...,42.10212,78.099655,21.900349,39.525997,28.447906,26.363533,36.39943,1,1,Cho Shmo
2022-11-12.22.38 Nickel Ranked Standard Win,3,3,0,1,1,0,192,False,0,253,...,45.823166,72.60137,27.39863,27.416456,31.864016,32.777897,21.049746,1,0,Cho Shmo
2022-10-23.20.09 O2 Ranked Standard Loss,1,8,0,4,1,1,214,False,0,268,...,32.377213,71.524,28.476004,34.32022,32.28752,31.46266,31.963469,1,1,Cho Shmo
2022-09-11.14.18 Ziggy. Ranked Standard Loss,3,9,3,2,2,0,667,True,100,283,...,38.3644,70.304405,29.6956,30.02925,34.14126,34.981033,32.51962,0,0,Cho Shmo
2022-09-08.17.44 Doc.^^ Ranked Standard Loss,3,7,1,2,1,0,266,False,33.333332,260,...,41.192116,71.83459,28.165417,32.572163,31.345037,28.410608,37.133865,0,1,Cho Shmo


In [42]:
boay00.to_csv('../data/boay00.csv')
cho_shmo.to_csv('../data/cho_shmo.csv')